# **Data Cleaning Notebook**

### Objectives

* Evaluate missing data.
* Clean data.

### Inputs

* inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv
* inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/inherited_houses.csv

### Outputs

* Test set: outputs/datasets/cleaned/test_set.csv
* Train set: outputs/datasets/cleaned/train_set.csv
* outputs/datasets/cleaned/clean_house_price_records.csv
* outputs/datasets/cleaned/clean_inherited_houses.csv

### Conclusions 

* Data Cleaning Pipeline.

### Additional Comments

* This file and its contents were inspired by and adapted from the Churnometer Walkthrough Project 2 and other lessons from Code Institute.  

---

### Change working directory

* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory

* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
from ydata_profiling import ProfileReport
from feature_engine.imputation import ArbitraryNumberImputer, CategoricalImputer
from sklearn.pipeline import Pipeline

### Load Collected Data

In [ ]:
df = pd.read_csv(f"inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv")
print(df.shape)

In [ ]:
df_inherited = pd.read_csv(f"inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/inherited_houses.csv")
print(df_inherited.shape)
df_inherited

### Data Exploration

* Identifying Columns with Missing Data:

In [ ]:
vars_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_missing_data

In [ ]:
if vars_missing_data:
   pandas_report = ProfileReport(df=df[vars_missing_data], minimal=True)
   pandas_report.to_notebook_iframe()
else:
   print("There are no variables with missing data.")

### Correlation and PPS Analysis

In this section, we aim to analyze the correlation between the target variable, SalePrice, and other features. We"ll use Pearson and Spearman correlation heatmaps, as well as a PPS heatmap, based on the PPS lesson to visualize these relationships.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps

%matplotlib inline

def heatmap_corr(df, threshold, figsize=(20,12), font_annot = 8):
  if len(df.columns) > 1:
    mask = np.zeros_like(df, dtype=bool)
    mask[np.triu_indices_from(mask)] = True
    mask[abs(df) < threshold] = True

    fig, axes = plt.subplots(figsize=figsize)
    sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                mask=mask, cmap="viridis", annot_kws={"size": font_annot}, ax=axes,
                linewidth=0.5
                     )
    axes.set_yticklabels(df.columns, rotation = 0)
    plt.ylim(len(df.columns),0)
    plt.show()


def heatmap_pps(df, threshold, figsize=(20,12), font_annot = 8):
    if len(df.columns) > 1:

      mask = np.zeros_like(df, dtype=bool)
      mask[abs(df) < threshold] = True

      fig, ax = plt.subplots(figsize=figsize)
      ax = sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                       mask=mask,cmap="rocket_r", annot_kws={"size": font_annot},
                       linewidth=0.05, linecolor="grey")
      
      plt.ylim(len(df.columns),0)
      plt.show()



def CalculateCorrAndPPS(df):
    # Filter out only numeric columns for correlation calculations
    numeric_df = df.select_dtypes(include=[np.number])

    # Calculate Spearman and Pearson correlations
    df_corr_spearman = numeric_df.corr(method="spearman")
    df_corr_pearson = numeric_df.corr(method="pearson")

    # Calculate the PPS matrix
    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(["x", "y", "ppscore"]).pivot(columns="x", index="y", values="ppscore")

    # Calculate PPS score statistics for thresholding
    pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(["ppscore"]).describe().T
    print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
    print(pps_score_stats.round(3))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(20,12), font_annot=8 ):

  print("\n")
  print("* Here I can analyze how the target variable for your ML models are correlated with other variables (features and target)")
  print("* Analyze multi colinearity, that is, how the features are correlated among themselves")

  print("\n")
  print("*** Heatmap: Spearman Correlation ***")
  print("It evaluates monotonic relationships between variables \n")
  heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

  print("\n")
  print("*** Heatmap: Pearson Correlation ***")
  print("It evaluates the linear relationship between two continuous variables \n")
  heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

  print("\n")
  print("*** Heatmap: Power Predictive Score (PPS) ***")
  print(f"PPS detects linear or non-linear relationships between two columns.\n"
        f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n")
  heatmap_pps(df=pps_matrix,threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)

Calculation of PPS:

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

Display at Heatmaps

In [ ]:
DisplayCorrAndPPS(df_corr_pearson=df_corr_pearson,
                  df_corr_spearman=df_corr_spearman,
                  pps_matrix=pps_matrix,
                  CorrThreshold=0.6,
                  PPS_Threshold=0.2,
                  figsize=(12, 10),
                  font_annot=10)

### Data Cleaning

### Assessing Missing Data Levels

* Custom function to display missing data levels in a DataFrame, it shows the absolute levels, relative levels and data type.

In [ ]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (pd.DataFrame(
                            data={"RowsWithMissingData": missing_data_absolute,
                                   "PercentageOfDataset": missing_data_percentage,
                                   "DataType": df.dtypes}
                                    )
                          .sort_values(by=["PercentageOfDataset"], ascending=False)
                          .query("PercentageOfDataset > 0")
                          )

    return df_missing_data

Check missing data levels for the collected dataset.

In [ ]:
EvaluateMissingData(df)

Create copy of house price dataframe:

In [ ]:
df_clean = df.copy()
print(df_clean.shape)

### Split Train and Test Set

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set, _, __ = train_test_split(
                                        df,
                                        df["SalePrice"],
                                        test_size=0.2,
                                        random_state=0)

print(f"train_set shape: {train_set.shape} \ntest_set shape: {test_set.shape}")

In [ ]:
df_missing_data = EvaluateMissingData(train_set)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

### Drop Variables

First we identify Variables with more than 80% missing data:

In [ ]:
threshold = 0.8
missing_fraction = train_set.isna().mean()
variables_to_drop = missing_fraction[missing_fraction > threshold].index.tolist()

print(f"Variables with more than 80% missing data: {variables_to_drop}")

Deeper analysis of missing Variables:

In [ ]:
df_enclosedporch = train_set.loc[train_set["EnclosedPorch"].notnull()]
df_enclosedporch[["EnclosedPorch", "SalePrice"]].plot(kind="scatter", x="EnclosedPorch", y="SalePrice")

df_wooddecksf = train_set.loc[train_set["WoodDeckSF"].notnull()]
df_wooddecksf[["WoodDeckSF", "SalePrice"]].plot(kind="scatter", x="WoodDeckSF", y="SalePrice")

df_wooddecksf["WoodDeckSF"].value_counts().sort_index(ascending=False).head(10)

### Dropping Variables with High Missing Values
After analyzing the dataset, we found that the following variables had more than 80% missing data:
- `EnclosedPorch`
- `WoodDeckSF`

These variables are unlikely to add significant value to our model due to the high proportion of missing data. Therefore, we decided to drop them from both the training and test datasets.

We then re-evaluated the dataset to check if any variables still have missing data and will proceed accordingly.

In [ ]:
from feature_engine.selection import DropFeatures
variables_to_drop = ["EnclosedPorch", "WoodDeckSF"]

imputer = DropFeatures(features_to_drop=variables_to_drop)
imputer.fit(train_set)
train_set, test_set = imputer.transform(train_set), imputer.transform(test_set)
train_set.head(3)

Drop features from inherited dataset:

In [ ]:
df_clean = imputer.transform(df_clean)

In [ ]:
null_variables = train_set.columns[train_set.isnull().any()].tolist()

Drop LotFrontage and MasVnrArea from the analysis due to their low correlation with SalePrice.

In [ ]:
train_set["LotFrontage"].value_counts().sort_index(ascending=False).head()
train_set["MasVnrArea"].value_counts().sort_index(ascending=False).head()

Impute Median value into null variables using MeanMedianImputer

In [ ]:
from feature_engine.imputation import MeanMedianImputer
variables = ["LotFrontage", "MasVnrArea"]
imputer = MeanMedianImputer(imputation_method="median", variables=variables)
imputer.fit(train_set)
train_set, test_set = imputer.transform(train_set), imputer.transform(test_set)

In [ ]:
df_clean = imputer.transform(df_clean)

In [ ]:
EvaluateMissingData(train_set)

We Remove EnclosedPorch and WoodDeckSF and imputed missing values for LotFrontage and MasVnrArea. These features no longer appear in our analysis list.

#### Second floor size

In [ ]:
train_set["2ndFlrSF"].value_counts().sort_index()

* 626 houses have 0 square feet on the second floor, indicating that these houses are single-story.

* There are 345 unique values in total, meaning houses with a second floor vary widely in size.

* The remaining values (besides 0) each appear only once, indicating a high variability in the second-floor area for houses that have it.

#### Bedrooms above grade

In [ ]:
train_set["BedroomAbvGr"].value_counts().sort_index()

* 3 bedrooms above ground is the most common, with 599 occurrences.

* 2 bedrooms is the second most common, appearing 266 times.

* 4 bedrooms is also relatively frequent, with 166 occurrences.

* Smaller counts are seen for 1-bedroom (32), 5-bedroom (16), 0-bedroom (4), and 6-bedroom (4) houses.

* There is one house with 8 bedrooms above ground, making it a rare case in the dataset.

This distribution highlights that most houses have 2–4 bedrooms above ground, with very few houses outside this range.

In [ ]:
pipeline = Pipeline([
      ( "2ndFlrSF",  ArbitraryNumberImputer(arbitrary_number=0,
                                                variables=["2ndFlrSF", "BedroomAbvGr"]) )
])
pipeline

pipeline.fit(train_set)
train_set, test_set = pipeline.transform(train_set), pipeline.transform(test_set)
df_clean = pipeline.transform(df_clean)
EvaluateMissingData(train_set)

- If `BsmtExposure` is "None", assume there is no basement, and fill with "None".
- For rows where `BsmtFinSF1` is 0 (indicating unfinished area), fill with "Unf".
- For remaining missing values, fill with "Unk" to indicate unknown status.

In [ ]:
train_set["BsmtFinType1"] = np.where(
    (train_set["BsmtExposure"] == "None") & (train_set["BsmtFinType1"].isnull()),
    "None",
    train_set["BsmtFinType1"]
)

train_set["BsmtFinType1"] = np.where(
    (train_set["BsmtFinSF1"] == 0) & (train_set["BsmtFinType1"].isnull()),
    "Unf",
    train_set["BsmtFinType1"]
)

pipeline_bsmtfintype1 = Pipeline([
    ("categorical_imputer", CategoricalImputer(imputation_method="missing", fill_value="Unk", variables=["BsmtFinType1"]))
])

pipeline_bsmtfintype1.fit(train_set)
train_set = pipeline_bsmtfintype1.transform(train_set)
EvaluateMissingData(train_set)


- If `BsmtFinType1` is "None" (indicating no basement), fill with "None" to indicate the absence of basement exposure.

In [ ]:
pipeline_bsmtexposure = Pipeline([
    ("categorical_imputer", CategoricalImputer(imputation_method="missing", fill_value="None", variables=["BsmtExposure"]))
])

pipeline_bsmtexposure.fit(train_set)
train_set = pipeline_bsmtexposure.transform(train_set)
EvaluateMissingData(train_set)


- If `GarageArea` is 0 (indicating no garage), fill with "None".
- For remaining missing values, assume the garage is unfinished, and fill with "Unf".

In [ ]:
train_set["GarageFinish"] = np.where(
    (train_set["GarageArea"] == 0) & (train_set["GarageFinish"].isnull()),
    "None",
    train_set["GarageFinish"]
)

pipeline_garagefinish = Pipeline([
    ("categorical_imputer", CategoricalImputer(imputation_method="missing", fill_value="Unf", variables=["GarageFinish"]))
])

pipeline_garagefinish.fit(train_set)
train_set = pipeline_garagefinish.transform(train_set)
EvaluateMissingData(train_set)

- The `GarageYrBlt` column represents the year the garage was built. If `GarageFinish` is "None" (indicating no garage), fill with 0 to mark the absence of a garage.

In [ ]:
pipeline_garageyrblt = Pipeline([
    ("arbitrary_number_imputer", ArbitraryNumberImputer(arbitrary_number=0, variables=["GarageYrBlt"]))
])

pipeline_garageyrblt.fit(train_set)
train_set = pipeline_garageyrblt.transform(train_set)
EvaluateMissingData(train_set)

We no longer have any missing values!

Function to se if datasets contain float columns:

In [ ]:

def list_float_columns(df, df_name):
    float_cols = df.select_dtypes(include="float").columns
    if len(float_cols) > 0:
        print(f"The following float columns are present in {df_name}:")
        for col in float_cols:
            print(f" - {col}")
    else:
        print(f"No float columns found in {df_name}.")

list_float_columns(df_clean, "df_clean")
list_float_columns(df_inherited, "df_inherited")
list_float_columns(train_set, "train_set")
list_float_columns(test_set, "test_set")


Convert float to int:

In [ ]:
def convert_floats_to_int(df):
    float_cols = df.select_dtypes("float").columns
    for col in float_cols:
        if (df[col] % 1 == 0).all():  
            df[col] = df[col].astype("int64")  
    return df

print("Original shape of df_clean:", df_clean.shape)
df_clean = convert_floats_to_int(df_clean)
print("df_clean after conversion:")
print(df_clean.select_dtypes("float").info())  

print("Original shape of df_inherited:", df_inherited.shape)
df_inherited = convert_floats_to_int(df_inherited)
print("df_inherited after conversion:")
print(df_inherited.select_dtypes("float").info())  

print("Original shape of train_set:", train_set.shape)
train_set = convert_floats_to_int(train_set)
print("train_set after conversion:")
print(train_set.select_dtypes("float").info())

print("Original shape of test_set:", test_set.shape)
test_set = convert_floats_to_int(test_set)
print("test_set after conversion:")
print(test_set.select_dtypes("float").info())

GarageYrBlt still contain a float. We convert it to an int:

In [ ]:
df_clean["GarageYrBlt"] = df_clean["GarageYrBlt"].fillna(0)
df_inherited["GarageYrBlt"] = df_inherited["GarageYrBlt"].fillna(0)
train_set["GarageYrBlt"] = train_set["GarageYrBlt"].fillna(0)
test_set["GarageYrBlt"] = test_set["GarageYrBlt"].fillna(0)

df_clean["GarageYrBlt"] = df_clean["GarageYrBlt"].astype(int)
df_inherited["GarageYrBlt"] = df_inherited["GarageYrBlt"].astype(int)
train_set["GarageYrBlt"] = train_set["GarageYrBlt"].astype(int)
test_set["GarageYrBlt"] = test_set["GarageYrBlt"].astype(int)

print("df_clean info after GarageYrBlt conversion:")
print(df_clean.info())
print("\n")

print("df_inherited info after GarageYrBlt conversion:")
print(df_inherited.info())
print("\n")

print("train_set info after GarageYrBlt conversion:")
print(train_set.info())
print("\n")

print("test_set info after GarageYrBlt conversion:")
print(test_set.info())


All missing values have been successfully handled, and no columns contain null values. Additionally, all relevant float columns have been converted to int, ensuring a consistent data format across the dataset for further analysis and modeling.

### Save Train and Test sets to csv

In [ ]:
import os
try:
  os.makedirs(name="outputs/datasets/cleaned") 
except Exception as e:
  print(e)

In [ ]:
train_set.to_csv("outputs/datasets/cleaned/train_set.csv", index=False)
test_set.to_csv("outputs/datasets/cleaned/test_set.csv", index=False)
df_clean.to_csv("outputs/datasets/cleaned/clean_house_price_records.csv", index=False)
df_inherited.to_csv("outputs/datasets/cleaned/clean_inherited_houses.csv", index=False)

### Conclusion and next step

The data cleaning process successfully handled all missing values and converted relevant float columns to integers, ensuring a consistent dataset ready for analysis. Next, we will proceed with Feature Engineering